# Single Excitation Configuration Interaction

In this notebook, we will be studying the effect of single excitation configuration interaction on the resuls obtained from constrained unrestricted hartree-fock. We will use the package psi4 for reference data and some basic methods. The UHF and CUHF code will come from the compChem package. We will begin this discussion with a theoretical approach to CIS and on the implementation in a normal UHF system. We will then proceed to extrapolate these findings to CUHF. The theory will be developed for RHF and then implemented for UHF, with the nessecary adjustments.

### Theoretical approach, findings and questions


When trying to make sanes of the theory, we need to make some findings. The sources here are 
- Szabo & Ostlund, _Modern Quantum Chemistry_
- Sherrill, _Derivation of the Configuration Interaction Singles (CIS) Method for Various Single Determinant References and Extensions to Include Selected Double Substitutions (XCIS)_
- the psi4  numpy tutorials, chapter 9


At first, let us take a look at the hamiltonian matrix in CIS. We know that we can express the wave function as Equation (1).
\begin{equation*} \tag{1} 
\Phi = c_0\Psi_0 + \sum_{ia} c^a_i\Phi^a_i
\end{equation*}
$\Psi_0$ is the normal HF wavefuntion. This function will be a better approximation for the exact wavefunction. However we will need to solve the schrödinger equation again, now using this new function. We will need to make a new hamiltonian, including all excited states as well as the ground state. There are a lot of possible combinations and one can imagine that it will be difficult to express let alone solve this matrix. However, we could also change our basis. When we use the ground state and the excited states as a basis, we find the following matrix (2).
\begin{equation*}\tag{2}
\hat{H} = \begin{bmatrix}\langle \Psi_0 |\hat{H}|\Psi_0 \rangle & 0 & 0 & \dots \\  
0 & \langle \Phi_i^a |\hat{H}|\Phi_i^a \rangle & \langle \Phi_i^b |\hat{H}|\Phi_i^a \rangle & \dots\\
0 & \langle \Phi_i^a |\hat{H}|\Phi_i^b \rangle & \langle \Phi_i^b |\hat{H}|\Phi_i^b \rangle & \dots\\
\end{bmatrix}
\end{equation*}
Overall the dimensions of this matrix will be the amount of occupied orbitals ($o$) times the amount of unoccupied orbitals ($u$), since for every occupied orbital $o$ there will be $u$ possible ways to create an excited state. However, note that in the represention that is being used here, $\Psi_0$ is also an eigenfunction of the hamiltonian, so we added an extra row and an extra collumn. The elements in the added rows and collums are all zero in accordance to Brillouin's theorem, which states that if $\Psi_0$ is a self consistent Hartree-Fock wave function, the overlap elements of the hamiltonian are zero for any single excited state. This means that the hamiltonian is block diagonal, with one very large block. Nevertheless, this is a step forward from it not being block diagonal. We can also pretty easily express these matrix elements. This can be seen in Equation (3).
\begin{equation*}\tag{3}
\langle \Phi_i^a |\hat{H}| \Phi_j^b \rangle = E_0\delta_{ij}\delta{ab} + F_{ab}\delta_{ij} - F_{ij}\delta_{ab} + \langle aj|| ib \rangle
\end{equation*}
Now that we have an expression for our hamiltonian, we can solve the eigenequations for it. The only thing left to figure out is what basis this hamiltonian is given in. Since it is made up out of the ground state and the excited states, these should all be eigenfunctions of the hamiltonian itself, wo the basis used is MO basis. We could start using the eigenfunctions of the fock operator at scf conditions (which is off course needed for Brillouin's theorem to hold) and start iterating from there. So we would calculate the eigenfunctions and the energy, use these to create a new operator and repeat. Alternatively we could substract $E_0$ from it, so that we get the change in energy with respect to the orginal HF energy. An additional note we can make is that $F_{ij} = \delta_{ij}\epsilon_i$ at scf conditions in MO basis, since the fock matrix should be diagonal there. This will simplify Equation (3) to Equation (4), in which we also substracted $E_0$.
\begin{equation*}\tag{4}
\langle \Phi_i^a |\hat{H} - E_0| \Phi_j^b \rangle =  (\epsilon_a - \epsilon_i)\delta_{ab}\delta_{ij} + \langle aj|| ib \rangle
\end{equation*}
Something we need to note is that the two electron integrals should be given in MO basis. These will need to be transformed. However we must note another aspect of importance. It seems illogical that electrons in an alpha spin orbital will only be excited to alpha orbitals. We will need to have all the orbitals open for excitation. Given that in the past we used the basisfunctions as a basis for both the alpha orbitals and the beta orbitals the total amount of orbitals needed will be twice the amount of basis functions. This means that we will need to "transform" all matrices that is double the size of normal basis. How can we do this? We could say that we just create a new Fock matrix that looks like Equation (5).
\begin{equation*}\tag{5}
\hat{F}_{new} = \begin{bmatrix} F_{\alpha} & 0 \\
0 & F_{\beta} \\
\end{bmatrix}
\end{equation*}
Now why is this a good idea? This fock matrix will have as eigenfunctions the eigenfunctions of both the $F_{\alpha}$ and the $F_{\beta}$ matrices. We also expect the C matrix to be block diagonal. This is off course logical, since the alpha orbitals will have no contributions to the beta orbitals, meaning that this block will be zero.

Questions at this point:
- Sherrill, equation (7): where does the - sign come from? :second quantisation
- Sherrill, equation (12): should this be $\epsilon_a - \epsilon_i$? :mistake on the webpage

### Codeblock 1: expressing the CIS Hamiltonian

In [1]:
# we need to import the packages and set some parameters
import psi4
import numpy as np
from scipy.linalg import eigh
from compChem.Hartree_Fock import Molecule
numpy_memory = 4
psi4.set_memory(int(5e8))



500000000

In [33]:
class CISMolecule():
    def __init__(self, molecule):
        """
        Will set up some variables we will need from the Molecule object

        input
        molecule: a Molecule object from the compChem package
        """
        self.id = molecule
        self.occupied = self.id.alpha + self.id.beta
        self.available = self.id.integrals.nbf()*2
        self.virtual = self.available - self.occupied
    

    def displayCISHamiltonian(self, spin):
        """displays the CIS hamiltonian in MO basis"""
        # getting the orbital energies
        assert spin == "alpha" or spin == "beta", f"{spin}: not a valid spin"
        epsilon_a, C_a = eigh(self.id.displayFockMatrix("alpha"), self.id.overlap)
        epsilon_b, C_b = eigh(self.id.displayFockMatrix("beta"), self.id.overlap)
        epsilon = epsilon_a + epsilon_b
        if spin == "alpha":
            C = C_a
        else:
            C = C_b

        # getting the two electron integrals in correct basis => we need it in MO basis
        tei = self.id.elrep # given in chemists notation
        tie_ao = tei.transpose(0, 2, 1, 3) - tei.transpose(0, 2, 3, 1) # accounts for both coulomb and exchange, switch to physisists notation
        tei_mo = np.einsum("pQRS,pP->PQRS", np.einsum("pqRS,qQ->pQRS", np.einsum("pqrS,rR->pqRS", np.einsum("pqrs,sS->pqrS", tie_ao, C, optimize=True), C, optimize=True), C, optimize=True), C, optimize=True)
        
        
        #getting the exitations
        exitations = []
        for orbital in range(self.occupied): # for every occupied orbital
            for another_orbital in range(self.occupied, self.available): # we can make an exitation to every virtual orbital
                exitations.append((orbital, another_orbital))
        
        # getting the hamiltonian
        dim = self.occupied*self.virtual
        H_cis = np.zeros((dim, dim))
        for row, exitation in enumerate(exitations):
            i, a = exitation
            for collumn, another_exitation in enumerate(exitations):
                j, b = another_exitation
                try:    
                    H_cis[row, collumn] = (epsilon[a] - epsilon[i])*(i == j)*(a == b) + tei_mo[a, j, i, b]
                except IndexError:
                    print(row, collumn, exitation, another_exitation)
                    break
        
# the current problem is the dimensions of the tei_mo => it needs to double in all directions (15, 15, 15, 15) => (30, 30, 30, 30)

        

The hamiltonian we are considering here is in fact the second block on the diagonal in Equation (2). We do this, because we need to get the eigenvalues of this hamiltonian and then simply add $E_0$ to that set to get the complete set of eigenvalues. If we substract $E_0$ from all the eigenvalues, we get the excitation energies

In [25]:
h3 = Molecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
E_0 = h3.iterator(mute=True)
E_0


/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:220: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"
/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:65: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":


-1.5062743202984428

In [ ]:
tei = h3.elrep

In [34]:
h3_cis = CISMolecule(h3)
h3_cis.occupied*h3_cis.virtual

81

In [35]:
h3_cis.displayCISHamiltonian("alpha")

0 12 (0, 3) (0, 15)
1 12 (0, 4) (0, 15)
2 12 (0, 5) (0, 15)
3 12 (0, 6) (0, 15)
4 12 (0, 7) (0, 15)
5 12 (0, 8) (0, 15)
6 12 (0, 9) (0, 15)
7 12 (0, 10) (0, 15)
8 12 (0, 11) (0, 15)
9 12 (0, 12) (0, 15)
10 12 (0, 13) (0, 15)
11 12 (0, 14) (0, 15)
12 0 (0, 15) (0, 3)
13 0 (0, 16) (0, 3)
14 0 (0, 17) (0, 3)
15 0 (0, 18) (0, 3)
16 0 (0, 19) (0, 3)
17 0 (0, 20) (0, 3)
18 0 (0, 21) (0, 3)
19 0 (0, 22) (0, 3)
20 0 (0, 23) (0, 3)
21 0 (0, 24) (0, 3)
22 0 (0, 25) (0, 3)
23 0 (0, 26) (0, 3)
24 0 (0, 27) (0, 3)
25 0 (0, 28) (0, 3)
26 0 (0, 29) (0, 3)
27 12 (1, 3) (0, 15)
28 12 (1, 4) (0, 15)
29 12 (1, 5) (0, 15)
30 12 (1, 6) (0, 15)
31 12 (1, 7) (0, 15)
32 12 (1, 8) (0, 15)
33 12 (1, 9) (0, 15)
34 12 (1, 10) (0, 15)
35 12 (1, 11) (0, 15)
36 12 (1, 12) (0, 15)
37 12 (1, 13) (0, 15)
38 12 (1, 14) (0, 15)
39 0 (1, 15) (0, 3)
40 0 (1, 16) (0, 3)
41 0 (1, 17) (0, 3)
42 0 (1, 18) (0, 3)
43 0 (1, 19) (0, 3)
44 0 (1, 20) (0, 3)
45 0 (1, 21) (0, 3)
46 0 (1, 22) (0, 3)
47 0 (1, 23) (0, 3)
48 0 (1, 24) (0,

Now the problem at hand is finding the energy. We need to find a way to get this concept into some kind of code for UHF. Let us begin with the stuf we will act upon. We will need to run the iteration for normal UHF and then use the matrices we get from here to implement CIS. When we calculate the eigenvalues of the hamiltonian, we will also get a coefficient matrix which expresses the eigenfunctions in MO basis. This can then be used to calculate the energy according to Equation (5).
\begin{equation*}\tag{5}
E_{CIS} = E_0 + 2\sum_{ia}c_0c_i^aF_{ia} + \sum_{iab} c_i^ac_i^bF_{ab} - \sum_{ija}c_i^ac_j^bF_{ij} + \sum_{ijab}c_i^ac_j^b \langle aj||ib \rangle
\end{equation*}
Note that the coefficients are known. Here we have a way to calculate the energy starting from the HF energy and adding some terms in. The only problem we face now is what matrices to use. The Fock matrices in the equation are MO Fock matrices. But does this equation hold for UHF? If not, how can we make it so? 